<a href="https://colab.research.google.com/github/uob-positron-imaging-centre/PEPT-Algorithms-RoPP/blob/main/SDM_RoPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank"  href="https://github.com/uob-positron-imaging-centre/pept"><img src="https://github.com/uob-positron-imaging-centre/misc-hosting/blob/master/logo.png?raw=true" style="height:200px; display: block; margin-left: auto; margin-right: auto;"/></a>

# Interactive PEPT Analysis Examples using *the Spherical Density Method*

> [1] Odo AE, Govender I, Buffler A, Franzidis JP. A PEPT algorithm for predefined positions of radioisotopes relative to the tracer particle. Applied Radiation and Isotopes. 2019 Sep 1;151:299-309.

---

#### Copyright 2021 the `pept` developers
##### Jupyter Notebook authored by Andrei Leonard Nicusan and Dr. Kit Windows-Yule for the "PEPT: A Comparative Review" paper, commissioned by the Reports on Progress in Physics journal

Licensed under the GNU License, Version 3.0 (the "License").

---


# 1. Introduction

Positron emission particle tracking (PEPT) is a powerful technique allowing the non-invasive, three-dimensional tracking of one or more radioactive 'tracer' particles through particulate, fluid or multiphase systems. It allows particle or fluid motion to be tracked with sub-millimetre accuracy and sub-millisecond temporal resolution and, due to its use of highly-penetrating 511keV gamma rays, can be used to probe the internal dynamics of even large, dense, optically opaque systems <sup>[[2]](https://www.sciencedirect.com/science/article/pii/016890029390864E) [[3]](https://www.sciencedirect.com/science/article/pii/S0263876208003341) [[4]](https://aip.scitation.org/doi/abs/10.1063/1.4983046@rsi.2017.IMGP2017.issue-1)</sup>. In light of its versatility both in terms of the scales and materials of particles which can be tracked <sup>[[5]](https://www.sciencedirect.com/science/article/pii/S1672251507001455)[[6]](https://www.sciencedirect.com/science/article/pii/S0168900206005341)</sup>, and the sizes and geometries of the systems which can be imaged <sup>[[7]](https://www.sciencedirect.com/science/article/pii/S0168900209001880) [[8]](https://www.sciencedirect.com/science/article/pii/S0029549316000273)</sup> , the technique has wide-ranging applicability in diverse scientific, industrial and biomedical applications.

PEPT is performed by radioactively labelling a particle with a positron-emitting radioisotope such as Fluorine-18 ($^{18}\mathrm{F}$) or Gallium-68 ($^{68}\mathrm{Ga}$), and using the back-to-back gamma rays produced by electron-positron annihilation events in and around the tracer to triangulate its spatial position. Each detected gamma ray represents a **line of response (LoR)** .

## 1.1. This Jupyter Notebook

This interactive Jupyter Notebook illustrates the main processing steps employed by the Spherical Density Method (SDM)<sup>[1]</sup> for radioactive tracer tracking, as described in the Reports on Progress in Physics "PEPT: A Comparative Review" paper.

An [example dataset](https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv) is used from an experiment run at the University of Birmingham Positron Imaging Centre using the ADAC Forté by Matthew Herald. It consists of a single 1 mm diameter MCC particle activated with Fluorine-18 radioactive tracer material inside a bubbling fluidised bed. The fluidised bed was filled with 90% sand and 10% MCC; air was fed into the bottom of the bed at a rate of 37 litres per minute at 3.5 bar. This dataset was chosen for its high quality captured lines of response, with the tracer still depicting the random particle motion that is inherent to bubbling fluidised beds - and typical in Lagrangian particle tracking.

The [`pept`](https://github.com/uob-positron-imaging-centre/pept) Python library is used for initialising and visualising PEPT data. While not required *per se* for illustrating PEPT algorithms' processing steps, it significantly reduces the amount of repetitive code and visual noise, allowing the reader to focus on the main conceptual procedures.

## 1.2. Running Code Cells
Select any code cell and click on the (▶) sign in the top-left of the cell's frame to run its code. 

In [ ]:
# First install the `pept` library using pip, Python's package manager
!pip install git+https://github.com/uob-positron-imaging-centre/pept.git

  Cloning https://github.com/uob-positron-imaging-centre/pept.git to /tmp/pip-req-build-ls5whb50
  Running command git clone -q https://github.com/uob-positron-imaging-centre/pept.git /tmp/pip-req-build-ls5whb50
     |████████████████████████████████| 6.4MB 12.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 235kB 47.6MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311648 sha256=4d87e4d8bf3dadc4de83aa14c349f6746dd4a65967408438fd251d05c674c2ca
  Stored in directory: /root/.cache/pip/wheels/42/63/fb/314ad6c3b270887a3ecb588b8e5aac50b0fad38ff89bb6dff2
Successfully built hdbscan
  Created wheel for pept: filename=pept-0.3.0-cp37-cp37m-linux_x86_64.whl size=4131202 sha256=02fe6cc008a624ccc7c536664a5b43f0d1d1f82265e06eee2710f7ca555606f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-s9u59wio/wheels/3e/b2/c9/8890c6e267a3f173

# 2. The Spherical Density Method

## 2.1. Read in Line of Response Data



In [ ]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 200,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors = np.insert(lors_raw, 6, head_separation, axis = 1)

# Print the line of response (LoR) data
lors

array([[0.000e+00, 1.900e+02, 1.687e+02, ..., 3.463e+02, 1.428e+02,
        6.000e+02],
       [1.000e-01, 2.437e+02, 1.676e+02, ..., 3.145e+02, 3.139e+02,
        6.000e+02],
       [1.000e-01, 1.941e+02, 4.100e+02, ..., 4.171e+02, 2.401e+02,
        6.000e+02],
       ...,
       [7.800e+00, 1.351e+02, 3.009e+02, ..., 4.366e+02, 1.870e+02,
        6.000e+02],
       [7.800e+00, 3.168e+02, 5.174e+02, ..., 4.195e+02, 2.100e+02,
        6.000e+02],
       [8.000e+00, 2.767e+02, 1.658e+02, ..., 2.938e+02, 2.024e+02,
        6.000e+02]])

In [ ]:
from pept.visualisation import PlotlyGrapher

grapher = PlotlyGrapher()
grapher.add_lines(lors)
grapher.show()

## 2.2. Calculate the LoR Distance Matrix

In [ ]:
def distance_lines(line1, line2):
    '''Calculate distance between two 3D lines `line1` and `line2`, each defined by two 3D points,
    so the lines are vectors formatted as [time, x1, y1, z1, x2, y2, z2].
    '''
    # Direction vectors:
    e1 = line1[4:7] - line1[1:4]
    e2 = line2[4:7] - line2[1:4]

    # Vector perpendicular to both lines
    n = np.cross(e1, e2)
    nd = np.linalg.norm(n)

    # If the lines are parallel, nd is zero
    if nd < 1e-8:
        e1 /= np.linalg.norm(e1)
        dist_vec = np.cross(e1, line2[1:4] - line1[1:4])
        return np.linalg.norm(dist_vec)

    dist = np.dot(n, line1[1:4] - line2[1:4]) / np.linalg.norm(n)
    return np.abs(dist)


distance_matrix = np.zeros((len(lors), len(lors)))
for i in range(len(lors)):
    for j in range(i + 1, len(lors)):
        distance_matrix[i, j] = distance_lines(lors[i], lors[j])
        distance_matrix[j, i] = distance_matrix[i, j]

distance_matrix

array([[  0.        ,  46.55313541, 102.72026331, ...,  84.19820107,
         57.37313057,  32.79265803],
       [ 46.55313541,   0.        ,  59.12484141, ...,   6.959341  ,
         96.5804409 ,  30.4307107 ],
       [102.72026331,  59.12484141,   0.        , ...,  49.81504488,
         19.60088111, 113.70490066],
       ...,
       [ 84.19820107,   6.959341  ,  49.81504488, ...,   0.        ,
         25.54578743,  53.06048451],
       [ 57.37313057,  96.5804409 ,  19.60088111, ...,  25.54578743,
          0.        , 123.70433647],
       [ 32.79265803,  30.4307107 , 113.70490066, ...,  53.06048451,
        123.70433647,   0.        ]])

## 2.3. Cluster LoRs Closer than the Tracer Radius

That is equivalent, in machine learning-speech, to single linkage clustering.

In [ ]:
from hdbscan import RobustSingleLinkage

tracer_radius = 0.5
min_lors_cluster = 5

single_linkage = RobustSingleLinkage(tracer_radius, min_lors_cluster, metric = "precomputed")
labels = single_linkage.fit_predict(distance_matrix)

labels

array([-1,  0, -1, -1,  0, -1, -1,  0,  0, -1, -1,  0,  0, -1, -1,  0,  0,
        0,  0, -1, -1,  0, -1,  0,  0, -1,  0,  0,  0, -1,  0, -1, -1,  0,
        0,  0,  0, -1, -1, -1,  0, -1, -1, -1,  0, -1, -1,  0,  0, -1, -1,
        0,  0,  0, -1,  0, -1,  0,  0, -1,  0, -1,  0, -1, -1,  0,  0, -1,
       -1, -1, -1,  0, -1, -1,  0, -1,  0,  0, -1, -1, -1,  0,  0, -1, -1,
       -1, -1,  0, -1,  0, -1,  0,  0, -1,  0, -1, -1,  0, -1,  0, -1,  0,
       -1, -1, -1, -1, -1, -1,  0,  0,  0, -1, -1, -1,  0,  0,  0, -1,  0,
       -1,  0,  0, -1,  0,  0,  0, -1,  0, -1,  0, -1, -1, -1, -1, -1,  0,
        0,  0,  0, -1,  0,  0,  0, -1,  0,  0,  0, -1, -1,  0,  0,  0,  0,
       -1, -1, -1, -1, -1,  0, -1,  0,  0, -1, -1,  0,  0,  0, -1, -1, -1,
        0, -1, -1, -1, -1, -1, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,
        0,  0, -1,  0,  0, -1,  0,  0, -1, -1, -1, -1, -1])

In [ ]:
# Colour-code LoRs based on the 
from pept.visualisation import PlotlyGrapher

labels_colour = np.repeat(labels, 3)

grapher = PlotlyGrapher()
grapher.add_lines(lors, color = labels_colour)
grapher.show()

## 2.4. Find Centroids of Clustered LoRs' Cutpoints 

In [ ]:
# For each label, collect the associated LoRs and compute the cutpoints
positions = []
for label in range(labels.max() + 1):
    cluster = lors[labels == label]
    cutpoints = pept.tracking.peptml.find_cutpoints(cluster, max_distance = np.inf)
    centre = cutpoints.mean(axis = 0)

    positions.append(centre)

positions = np.array(positions)
positions

array([[  3.94735919, 279.75777575, 243.88478195, 315.08413242]])

In [ ]:
grapher = PlotlyGrapher(
    cols = 2,
    subplot_titles = ["Clustered LoRs", "Clustered LoRs' Cutpoints and Centroid"],
)

grapher.add_lines(lors, color = labels_colour)
grapher.add_points(cutpoints, col = 2)
grapher.add_points(positions, size = 10, col = 2)

grapher.show()

# 3. Complete SDM Code

In [ ]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 1000,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors = np.insert(lors_raw, 6, head_separation, axis = 1)

# Print the line of response (LoR) data
lors

array([[0.000e+00, 1.900e+02, 1.687e+02, ..., 3.463e+02, 1.428e+02,
        6.000e+02],
       [1.000e-01, 2.437e+02, 1.676e+02, ..., 3.145e+02, 3.139e+02,
        6.000e+02],
       [1.000e-01, 1.941e+02, 4.100e+02, ..., 4.171e+02, 2.401e+02,
        6.000e+02],
       ...,
       [3.560e+01, 4.596e+02, 8.440e+01, ..., 1.923e+02, 2.384e+02,
        6.000e+02],
       [3.570e+01, 2.325e+02, 5.463e+02, ..., 3.540e+02, 7.670e+01,
        6.000e+02],
       [3.570e+01, 4.584e+02, 5.334e+02, ..., 4.549e+02, 7.020e+01,
        6.000e+02]])

In [ ]:
# Use the Spherical Density Method (SDM) to track moving tracer
from hdbscan import RobustSingleLinkage

tracer_radius = 0.5

sample_start = 0
sample_size = 200

positions = []


def distance_lines(line1, line2):
    '''Calculate distance between two 3D lines `line1` and `line2`, each defined by two 3D points,
    so the lines are vectors formatted as [time, x1, y1, z1, x2, y2, z2].
    '''
    # Direction vectors:
    e1 = line1[4:7] - line1[1:4]
    e2 = line2[4:7] - line2[1:4]

    # Vector perpendicular to both lines
    n = np.cross(e1, e2)
    nd = np.linalg.norm(n)

    # If the lines are parallel, nd is zero
    if nd < 1e-8:
        e1 /= np.linalg.norm(e1)
        dist_vec = np.cross(e1, line2[1:4] - line1[1:4])
        return np.linalg.norm(dist_vec)

    dist = np.dot(n, line1[1:4] - line2[1:4]) / np.linalg.norm(n)
    return np.abs(dist)


while sample_start + sample_size < len(lors):
    sample = lors[sample_start:sample_start + sample_size]

    distance_matrix = np.zeros((len(sample), len(sample)))
    for i in range(len(sample)):
        for j in range(i + 1, len(sample)):
            distance_matrix[i, j] = distance_lines(sample[i], sample[j])
            distance_matrix[j, i] = distance_matrix[i, j]

    min_lors_cluster = 5

    single_linkage = RobustSingleLinkage(tracer_radius, min_lors_cluster, metric = "precomputed")
    labels = single_linkage.fit_predict(distance_matrix)

    for label in range(labels.max() + 1):
        cluster = sample[labels == label]
        cutpoints = pept.tracking.peptml.find_cutpoints(cluster, max_distance = np.inf)
        centre = cutpoints.mean(axis = 0)

        positions.append(centre)

    sample_start += sample_size

positions = np.array(positions)
positions

array([[  3.94735919, 279.75777575, 243.88478195, 315.08413242],
       [ 11.75650191, 278.81017366, 243.83416368, 315.49590416],
       [ 19.01297622, 280.11003014, 243.72010854, 316.39736445],
       [ 26.70645841, 280.55534061, 244.24580937, 317.66650764]])

In [ ]:
grapher = PlotlyGrapher(
    cols = 3,
    subplot_titles = [
        "Last Sample's Clustered LoRs",
        "Last Sample's Clustered LoRs' Cutpoints",
        "Tracer Positions",
    ],
)

labels_colour = np.repeat(labels, 3)

grapher.add_lines(lors[sample_start - sample_size:sample_start], color = labels_colour)
grapher.add_points(cutpoints, col = 2)
grapher.add_points(positions, col = 3)

grapher.show()